## 1. Getting all the imports

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPool1D, Embedding, Input
import os
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
import json
import itertools
from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

## 2. Loading data

Read the data that was stored in the JSON format.

In [3]:
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


def load_file():
    with open('./data/arxiv-metadata-oai-snapshot.json') as f:
        for line in f:
            yield json.loads(line)


metadata = load_file()

subset = itertools.islice(metadata, 100000)

df = pd.DataFrame(subset)
df.head(5)

./data\arxiv-metadata-oai-snapshot.json


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


Similar to what we did for the "bow_model.ipynb" notebook, we are only looking into the 6 major categories for now

In [4]:
df_cat = df[df['categories'].isin(['hep-ph', 'quant-ph', 'astro-ph', 'hep-th', 'gr-qc', 'cond-mat.mtrl-sci'])] # high energy physics/quantum physics
df_cat['categories'].value_counts()

df_cat = pd.get_dummies(df_cat, columns=['categories'])

X = df_cat['abstract']

# Dependent Variables
y = df_cat[['categories_astro-ph', 'categories_gr-qc',
            'categories_hep-ph', 'categories_hep-th', 'categories_quant-ph', 'categories_cond-mat.mtrl-sci']]

train_text, test_text, train_labels, test_labels = train_test_split(X, y, test_size=0.3, random_state=123)

## 3. Text preprocessing

1. We first initialize the tokenizer to split the words into lists of tokens. The num_words is set to 20000, which is the maximum number to keep.
2. Using 'fit_on_texts' function to update the internal vocabulary based on the list of texts. 
3. Calling the 'texts_to_sequences' function which transforms each text in texts to a sequence of integers

In [5]:
# Initializing the class
tokenizer = Tokenizer(num_words = MAX_NUM_WORDS)
# Updates internal vocabulary based on a list of texts.
tokenizer.fit_on_texts(train_text)

# Transforms each text in texts to a sequence of integers.
train_sequences = tokenizer.texts_to_sequences(train_text)
test_sequences = tokenizer.texts_to_sequences(test_text)

word_index = tokenizer.word_index
print("Length of word Index:", len(word_index))
print("First 5 elements in the word_index dictionary:", dict(list(word_index.items())[0: 5]) )
print("First abstract text in training set:\n", train_sequences[0])

Length of word Index: 46580
First 5 elements in the word_index dictionary: {'the': 1, 'of': 2, 'and': 3, 'a': 4, 'in': 5}
First comment text in training set:
 [7, 885, 72, 2792, 183, 1, 1338, 1921, 4509, 602, 13, 3816, 916, 39, 408, 1, 1013, 541, 1339, 496, 4195, 1932, 950, 39, 9, 1, 2215, 123, 3722, 91, 2, 1542, 176, 974, 4, 3, 64, 135, 17, 85, 17, 2617, 1719, 7, 682, 1013, 100, 255, 1104, 1961, 404, 2, 4346, 649, 3, 43, 25, 55, 341, 179, 180, 39, 16, 1, 7391, 219, 53, 2, 1, 1463, 1104, 653, 2829, 6, 1714, 5, 30, 2670, 7, 885, 1, 6069, 2, 1, 3722, 1815, 4, 64, 3, 4, 22, 1434, 5, 5022, 916, 3, 496, 4195, 39, 18, 90, 650, 3, 2041, 1, 4594, 3, 2378, 182, 7, 36, 473, 1, 515, 46, 82, 182, 3, 1, 271, 46, 82, 182, 77, 17, 4, 118, 2, 194, 42, 916, 3, 950, 141, 565, 13, 3722]


Now that we have tokenized the comment texts, we need to pad the sentences to make all the sentences of equal length. This is because for DL model inputs, we should have a fixed length of data inputs. 


In [6]:
# Pad tokenized sequences
trainvalid_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print("Shape of padded sequence list:\n", trainvalid_data.shape)
print("First abstract text in training set - 0 for padding - only last 50 sequences as the rest are paddings:\n", trainvalid_data[0][-50:])

Shape of padded sequence list:
 (23494, 1000)
First abstract text in training set - 0 for padding - only last 50 sequences as the rest are paddings:
 [  64    3    4   22 1434    5 5022  916    3  496 4195   39   18   90
  650    3 2041    1 4594    3 2378  182    7   36  473    1  515   46
   82  182    3    1  271   46   82  182   77   17    4  118    2  194
   42  916    3  950  141  565   13 3722]


## 4. Data modeling

CNN 1D model is used to classify the articles. A standard model for document classification is to use an Embedding layer as input, followed by a one-dimensional convolutional neural network, pooling layer, and then a prediction output layer. We used 1 embedding layer, 3 sets of Convolution and Pooling layer and 2 sets of Dense layer. These hyperparameters can be fine-tuned to improve the performance

In [7]:
# Using 1D CNN model
cnn_model = Sequential()
cnn_model.add(Embedding(MAX_NUM_WORDS, 128))
cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = "relu"))
cnn_model.add(MaxPooling1D(pool_size = 5))
cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = "relu"))
cnn_model.add(MaxPooling1D(pool_size = 5))
cnn_model.add(Conv1D(filters = 128, kernel_size = 5, activation = "relu"))
cnn_model.add(GlobalMaxPool1D())
cnn_model.add(Dense(units = 128, activation = 'relu'))
cnn_model.add(Dense(units = 6, activation = 'sigmoid'))

print(cnn_model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         2560000   
_________________________________________________________________
conv1d (Conv1D)              (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 128)         0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         82048     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0

In [8]:
# Configures the model for training.
cnn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["AUC", "accuracy"])

# Split the dataset into train and validation set for training and evaludating the model
X_train, X_val, y_train, y_val = train_test_split(trainvalid_data, train_labels, shuffle=True, random_state=123)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

# Trains the model for a fixed number of epochs (iterations on a dataset)
history = cnn_model.fit(X_train, y_train, batch_size=128, epochs=20, validation_data=(X_val, y_val))


(17620, 1000) (17620, 6) (5874, 1000) (5874, 6)
Epoch 1/20
138/138 [==============================] - 201s 1s/step - loss: 0.2835 - auc: 0.8698 - accuracy: 0.5763 - val_loss: 0.2156 - val_auc: 0.9269 - val_accuracy: 0.6442
Epoch 2/20
138/138 [==============================] - 214s 2s/step - loss: 0.1889 - auc: 0.9487 - accuracy: 0.7019 - val_loss: 0.1631 - val_auc: 0.9643 - val_accuracy: 0.7930
Epoch 3/20
138/138 [==============================] - 210s 2s/step - loss: 0.0948 - auc: 0.9879 - accuracy: 0.8898 - val_loss: 0.0795 - val_auc: 0.9911 - val_accuracy: 0.9067
Epoch 4/20
138/138 [==============================] - 220s 2s/step - loss: 0.0375 - auc: 0.9977 - accuracy: 0.9591 - val_loss: 0.0846 - val_auc: 0.9893 - val_accuracy: 0.9162
Epoch 5/20
138/138 [==============================] - 192s 1s/step - loss: 0.0192 - auc: 0.9992 - accuracy: 0.9809 - val_loss: 0.1012 - val_auc: 0.9841 - val_accuracy: 0.9145
Epoch 6/20
138/138 [==============================] - 188s 1s/step - loss: 0.

In [9]:
# Use the CNN model to output probabilities on test data
y_preds = cnn_model.predict(test_data)

y_vals = test_labels.to_numpy()
# Model Probabilities for class 1 of each of the target variables
# y_preds = np.transpose(np.array(cnn_model.predict_proba(X_val))[:, :, 1])

y_preds_2 = np.argmax(y_preds, axis=1)
y_test_2 = np.argmax(y_vals, axis=1)
print(f"The Classification report: \n")
print(classification_report(y_preds_2, y_test_2))

The Classification report: 

              precision    recall  f1-score   support

           0       0.97      0.97      0.97      4829
           1       0.83      0.53      0.65       974
           2       0.88      0.95      0.91      1499
           3       0.78      0.82      0.80      1056
           4       0.79      0.94      0.86      1040
           5       0.88      0.87      0.87       672

    accuracy                           0.90     10070
   macro avg       0.86      0.85      0.84     10070
weighted avg       0.90      0.90      0.89     10070



In [10]:
def calculate_roc_auc(y_test, y_pred):
    aucs = []
    # Calculate the ROC-AUC for each of the target column
    for col in range(y_test.shape[1]):
        aucs.append(roc_auc_score(y_test[:,col],y_pred[:,col]))
    return aucs

# Calculate Mean of the ROC-AUC
mean_auc = mean(calculate_roc_auc(y_vals, y_preds))

In [11]:
mean_auc

0.9863582874569641